In [ ]:
import numpy as np
import pylab
%matplotlib inline
import scipy.ndimage as ndimage
from PIL import ImageDraw, ImageFont, Image
import pickle
#import tensorflow as tf
#from tensorflow import keras
import keras
#from keras.datasets import mnist
from keras import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils import np_utils

In [ ]:
trainingfile=open("Pkls/TrainingFile.pkl",'rb')
((Images,Labels),(TestImages,TestLabels))=pickle.load(trainingfile)
trainingfile.close()

In [ ]:
pylab.imshow(Images[1],cmap='binary')
pylab.colorbar()
pylab.title(Labels[1])
pylab.show()

pylab.imshow(TestImages[1],cmap='binary')
pylab.colorbar()
pylab.title(TestLabels[1])
pylab.show()

XSize=len(Images[1])
YSize=len(Images[1][0])


In [ ]:
nclasses=26
ImgSize=len(Images[0])*len(Images[0][0])
X_train = Images.reshape(len(Images), XSize,YSize,1)
Y_train = np_utils.to_categorical(Labels, nclasses)
X_test = TestImages.reshape(len(TestImages), XSize,YSize,1)
Y_test = np_utils.to_categorical(TestLabels, nclasses)



In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(XSize,YSize,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(nclasses, activation='softmax'))

In [ ]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.topkacc = []
        self.loss = []
        self.valid_acc=[]
        self.valid_topkacc=[]
        self.valid_loss=[]
    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        self.topkacc.append(logs.get('top_k_categorical_accuracy'))
        self.loss.append(logs.get('loss'))
        validloss, validtopkacc, validacc= self.model.evaluate(X_test, Y_test, verbose=0)
        self.valid_loss.append(validloss)
        self.valid_acc.append(validacc)
        self.valid_topkacc.append(validtopkacc)
history = AccuracyHistory()

In [ ]:
model.compile(optimizer=keras.optimizers.SGD(lr=0.01), 
              loss='categorical_crossentropy',
              metrics=['top_k_categorical_accuracy','accuracy'])


In [ ]:
traininglog=model.fit(X_train, Y_train, epochs=10,batch_size=128, callbacks=[history])

In [ ]:
history.valid_acc

In [ ]:
pylab.figure(figsize=(4,4))
pylab.plot(history.acc,'--',label='Accuracy, Training',color='DarkRed')
pylab.plot(history.valid_acc,label='Accuracy, Validation',color='DarkRed')

pylab.plot(history.topkacc,'--',label='TopK Accuracy, Training',color='black')
pylab.plot(history.valid_topkacc,label='TopK Accuracy, Validation',color='black')


pylab.xlabel("Epoch")
pylab.ylabel("Fig. of Merit")
pylab.legend(loc='lower right')
pylab.savefig("Plots/NetworkAcc.png",dpi=250)

In [ ]:
pylab.figure(figsize=(4,4))
pylab.plot(history.loss,'--,',label='Loss, Training',color='DarkBlue')
pylab.plot(history.valid_loss,label='Loss, Validation',color='DarkBlue')

pylab.xlabel("Epoch")
pylab.ylabel("Fig. of Merit")
pylab.legend(loc='upper right')
pylab.savefig("Plots/NetworkLoss.png",dpi=250)

In [ ]:
predictions = model.predict(X_test)

In [ ]:


model.save("Network/TrainedModelFuzzy.h5")


In [ ]:
def LabelToLetter(l):
    return chr(65+l)
Letters=[]
Indices=np.arange(0,26)
for i in Indices:
    Letters.append(LabelToLetter(i))



In [ ]:
TheMatrix=np.ndarray(shape=(26,26))
TheMatrix=np.zeros_like(TheMatrix)
for p in range(0,len(predictions)):
    for c in range(0,len(predictions[p])):
        TheMatrix[TestLabels[p]][c]+=predictions[p][c]
TheMatrix=TheMatrix/len(predictions)*26

In [ ]:
def PlotCheckPrediction(predictions, x):
    pred=predictions[x]
    f, (ax1, ax2) = pylab.subplots(1, 2, figsize=(12,4))
    ax1.hist(Indices,weights=pred,bins=np.arange(-0.5,26.5,1))
   # ax1.hist(Indices,weights=pred*TheMatrix[:,],bins=np.arange(-0.5,26.5,1),histtype='step')


    ax2.imshow(TestImages[x],cmap='binary')
    ax2.set_title("Truth = "+str(LabelToLetter(TestLabels[x])))
    ax1.set_title("Best Guess = "+str(LabelToLetter(np.argmax(pred))))
    ax1.set_xticks(Indices)
    ax1.set_xticklabels(Letters)
    ax1.set_ylabel("CNN Score")
    ax1.set_xlim(-0.5,25.5)
    ax1.set_ylim(0,1)
    ax1.grid()
    pylab.show()

In [ ]:
for i in range(0,30):
    PlotCheckPrediction(predictions,i)

In [ ]:
sum(predictions[4])

In [ ]:
TrainingFreqs=np.zeros(26)
TheMatrix=np.ndarray(shape=(26,26))
TheMatrix=np.zeros_like(TheMatrix)
for p in range(0,len(predictions)):
    TrainingFreqs[TestLabels[p]]+=1.
    for c in range(0,len(predictions[p])):
        TheMatrix[TestLabels[p]][c]+=predictions[p][c]
TransferFun=np.transpose(TheMatrix)/TrainingFreqs
np.savetxt("./TransferMatrix.txt",TransferFun)

In [ ]:
pylab.figure(figsize=(4,4),dpi=200)
pylab.imshow(TransferFun)
pylab.xlabel("True letter")
pylab.ylabel("Average Score")
pylab.colorbar()


In [ ]:
TrainingFreqs

In [ ]:
MaxMatrix=np.ndarray(shape=(26,26))
MaxMatrix=np.zeros_like(TheMatrix)
for p in range(0,len(predictions)):
    m=np.argmax(predictions[p])
    MaxMatrix[TestLabels[p]][m]+=1.
TransferFunMax=np.transpose(MaxMatrix)/TrainingFreqs



In [ ]:
sum(MaxMatrix)

In [ ]:
import matplotlib as mpl
pylab.figure(figsize=(4,4),dpi=200)
pylab.imshow(TransferFunMax)
pylab.xlabel("True letter")
pylab.ylabel("Frequency Of Best Fit")
pylab.colorbar()
pylab.clim(0,1)


In [ ]:
sum(np.transpose(MaxMatrix))

In [ ]:
sum(MaxMatrix)

In [ ]:
pylab.plot(TrainingFreqs,'o')

In [ ]:
80*26